In [1]:
import pandas as pd
import numpy as np
from custom_function import DicodingProject1 as DP1
from tensorflow.keras.models import load_model  # type: ignore
from sklearn.metrics import accuracy_score

2025-04-12 11:10:39.679686: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-12 11:10:39.702314: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744431039.720452   54011 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744431039.725849   54011 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744431039.742407   54011 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

#### Explore Dataset Labelled using Hartmann DistilRoberta-base

In [2]:
DATASET = 'hartmann'
df_poem = pd.read_csv(f'./dataset/labelled_poem_{DATASET}.csv')
df_poem['label'].value_counts()

label
sadness     6431
fear        3920
anger       2833
joy         2550
neutral     1025
surprise     544
disgust      109
Name: count, dtype: int64

In [3]:
# Treshold digunakan untuk memfilter dari score yang lebih rendah (Baca ATTENTION pada file labelling.ipynb)
threshold = np.quantile(df_poem['score'], 0.4)
df_poem_filtered = df_poem[df_poem['score']>=threshold]
f'Set threshold {round(threshold, 5)}', df_poem_filtered['label'].value_counts()

('Set threshold 0.75023',
 label
 sadness     4352
 fear        2637
 anger       1608
 joy         1457
 neutral      242
 surprise     127
 disgust       24
 Name: count, dtype: int64)

In [4]:
# df_poem_filtered = df_poem_filtered[(df_poem_filtered['label']!='neutral') & (df_poem_filtered['label']!='disgust') & (df_poem_filtered['label']!='surprise')]
df_poem_filtered = df_poem.copy()
df_poem_filtered['label'].value_counts(), f'Banyak data adalah {df_poem_filtered['label'].value_counts().sum()}'

(label
 sadness     6431
 fear        3920
 anger       2833
 joy         2550
 neutral     1025
 surprise     544
 disgust      109
 Name: count, dtype: int64,
 'Banyak data adalah 17412')

From 7 labels, only used 4 because the imbalance class distribution are too big

In [5]:
TEST_SIZES = [0.1]
EPOCHS = 1000
MODES = ['lstm']
BATCH_SIZE = 32
CLASS = df_poem_filtered['label'].nunique()
MAX_LENGTH = 256

for TEST_SIZE in TEST_SIZES:
    for MODE in MODES:
        print('')
        kelas = DP1(TEST_SIZE, EPOCHS, MODE, CLASS, DATASET)
        X_train_HF, X_val_HF, y_train_HF, y_val_HF = kelas.prepareDf(df_poem_filtered)
        X_train_HF_keras, X_val_HF_keras, vocab_size, embedding_matrix = kelas.gloveTokenizer(X_train_HF, X_val_HF)
        train_dataset_HF = kelas.createDataset(X_train_HF_keras, y_train_HF)
        val_dataset_HF = kelas.createDataset(X_val_HF_keras, y_val_HF)
        model = kelas.buildModel(MODE, vocab_size, embedding_matrix)
        model.fit(
            train_dataset_HF,
            epochs = EPOCHS,
            batch_size = BATCH_SIZE,
            validation_data = val_dataset_HF,
            callbacks=[kelas.checkpointCallback(), kelas.earlyStopping()]
        )
        best_model = load_model(f'./model/best_model_{DATASET}_{TEST_SIZE}_{MODE}.keras')
        result = best_model.predict(X_val_HF_keras)
        predicted_labels = np.argmax(result, axis=1)
        accuracy = accuracy_score(y_val_HF, predicted_labels)
        print(f'Akurasi untuk skema {DATASET}, split test {TEST_SIZE*100}%, model {MODE} adalah {accuracy*100}%')
        with open('4.1. log_testing_result.txt', 'a') as f:
            f.write(f'Akurasi untuk skema {DATASET}, split test {TEST_SIZE*100}%, model {MODE} adalah {accuracy*100}% \n')

I0000 00:00:1744429455.164571   44022 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Epoch 1/1000


I0000 00:00:1744429459.898685   45462 cuda_dnn.cc:529] Loaded cuDNN version 90300


634/634 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.1870 - loss: 2.1442
Epoch 1: val_accuracy improved from -inf to 0.35829, saving model to ./model/best_model_hartmann_0.1_lstm.keras
634/634 ━━━━━━━━━━━━━━━━━━━━ 34s 45ms/step - accuracy: 0.1871 - loss: 2.1439 - val_accuracy: 0.3583 - val_loss: 1.6281
Epoch 2/1000
633/634 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.3832 - loss: 1.5837
Epoch 2: val_accuracy improved from 0.35829 to 0.51622, saving model to ./model/best_model_hartmann_0.1_lstm.keras
634/634 ━━━━━━━━━━━━━━━━━━━━ 29s 45ms/step - accuracy: 0.3833 - loss: 1.5834 - val_accuracy: 0.5162 - val_loss: 1.2205
Epoch 3/1000
634/634 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.5250 - loss: 1.2203
Epoch 3: val_accuracy improved from 0.51622 to 0.60151, saving model to ./model/best_model_hartmann_0.1_lstm.keras
634/634 ━━━━━━━━━━━━━━━━━━━━ 29s 45ms/step - accuracy: 0.5250 - loss: 1.2202 - val_accuracy: 0.6015 - val_loss: 1.0084
Epoch 4/1000
634/634 ━━━━━━━━━━━━━━━━━━━━ 0s 40

---

#### Explore Dataset Labelled using Savani Distilbert-base

In [2]:
DATASET = 'savani'
df_poem = pd.read_csv(f'./dataset/labelled_poem_{DATASET}.csv')
df_poem['label'].value_counts()

label
joy         6524
sadness     4811
anger       3297
fear        1527
love        1179
surprise      74
Name: count, dtype: int64

In [3]:
threshold = np.quantile(df_poem['score'], 0.32)
df_poem_filtered = df_poem[df_poem['score']>=threshold]
f'Set threshold {round(threshold, 5)}', df_poem_filtered['label'].value_counts()

('Set threshold 0.6674',
 label
 joy         4841
 sadness     3331
 anger       1873
 fear         928
 love         813
 surprise      54
 Name: count, dtype: int64)

In [4]:
# df_poem_filtered = df_poem_filtered[(df_poem_filtered['label']!='fear') & (df_poem_filtered['label']!='disgust') & (df_poem_filtered['label']!='love') & (df_poem_filtered['label']!='surprise')]
df_poem_filtered =  df_poem.copy()
df_poem_filtered['label'].value_counts(), f'Banyak data adalah {df_poem_filtered['label'].value_counts().sum()}'
# Banyak data sudah lebih dari 10.000. Amann

(label
 joy         6524
 sadness     4811
 anger       3297
 fear        1527
 love        1179
 surprise      74
 Name: count, dtype: int64,
 'Banyak data adalah 17412')

From 6 labels, only used 3 because the imbalance class distribution are too big

In [5]:
TEST_SIZES = [0.1]
EPOCHS = 1000
MODES = ['lstm']
BATCH_SIZE = 32
CLASS = df_poem_filtered['label'].nunique()
MAX_LENGTH = 256

for TEST_SIZE in TEST_SIZES:
    for MODE in MODES:
        print(f'Skenario Dataset {DATASET} : {TEST_SIZE} mode {MODE}')
        kelas = DP1(TEST_SIZE, EPOCHS, MODE, CLASS, DATASET)
        X_train_HF, X_val_HF, y_train_HF, y_val_HF = kelas.prepareDf(df_poem_filtered)
        X_train_HF_keras, X_val_HF_keras, vocab_size, embedding_matrix = kelas.gloveTokenizer(X_train_HF, X_val_HF)
        train_dataset_HF = kelas.createDataset(X_train_HF_keras, y_train_HF)
        val_dataset_HF = kelas.createDataset(X_val_HF_keras, y_val_HF)
        model = kelas.buildModel(MODE, vocab_size, embedding_matrix)
        model.fit(
            train_dataset_HF,
            epochs = EPOCHS,
            batch_size = BATCH_SIZE,
            validation_data = val_dataset_HF,
            callbacks=[kelas.checkpointCallback(), kelas.earlyStopping()]
        )
        best_model = load_model(f'./model/best_model_{DATASET}_{TEST_SIZE}_{MODE}.keras')
        result = best_model.predict(X_val_HF_keras)
        predicted_labels = np.argmax(result, axis=1)
        accuracy = accuracy_score(y_val_HF, predicted_labels)
        print(f'Akurasi untuk skema {DATASET}, split test {TEST_SIZE*100}%, model {MODE} adalah {accuracy*100}%')
        with open('4.1. log_testing_result.txt', 'a') as f:
            f.write(f'Akurasi untuk skema {DATASET}, split test {TEST_SIZE*100}%, model {MODE} adalah {accuracy*100}% \n')

Skenario Dataset savani : 0.1 mode lstm


I0000 00:00:1744431245.499683   54011 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Epoch 1/1000


I0000 00:00:1744431249.237690   54902 cuda_dnn.cc:529] Loaded cuDNN version 90300


550/551 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.1933 - loss: 2.0535
Epoch 1: val_accuracy improved from -inf to 0.34891, saving model to ./model/best_model_savani_0.1_lstm.keras
551/551 ━━━━━━━━━━━━━━━━━━━━ 30s 46ms/step - accuracy: 0.1934 - loss: 2.0529 - val_accuracy: 0.3489 - val_loss: 1.6117
Epoch 2/1000
551/551 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.3277 - loss: 1.6260
Epoch 2: val_accuracy improved from 0.34891 to 0.44291, saving model to ./model/best_model_savani_0.1_lstm.keras
551/551 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.3278 - loss: 1.6259 - val_accuracy: 0.4429 - val_loss: 1.3450
Epoch 3/1000
550/551 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.4321 - loss: 1.3606
Epoch 3: val_accuracy improved from 0.44291 to 0.50651, saving model to ./model/best_model_savani_0.1_lstm.keras
551/551 ━━━━━━━━━━━━━━━━━━━━ 24s 44ms/step - accuracy: 0.4322 - loss: 1.3604 - val_accuracy: 0.5065 - val_loss: 1.1983
Epoch 4/1000
550/551 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/ste

#### Explore Dataset Labelled using Deepseek Model

In [2]:
DATASET = 'deepseek'
df_poem = pd.read_csv(f'./dataset/labelled_poem_{DATASET}.csv')
df_poem['label'].value_counts()

label
other      5687
sadness    4113
joy        3733
hope       2605
love       1274
Name: count, dtype: int64

In [ ]:
# df_poem_filtered = df_poem[df_poem['label'] != 'other']
df_poem_filtered = df_poem.copy()
df_poem_filtered['label'].value_counts()

label
other      5687
sadness    4113
joy        3733
hope       2605
love       1274
Name: count, dtype: int64

In [4]:
TEST_SIZES = [0.1] # ,  0.2 0.1
EPOCHS = 1000
MODES = ['lstm'] # , 'gru'
BATCH_SIZE = 32
CLASS = df_poem_filtered['label'].nunique()
MAX_LENGTH = 256

for TEST_SIZE in TEST_SIZES:
    for MODE in MODES:
        print('')
        kelas = DP1(TEST_SIZE, EPOCHS, MODE, CLASS, DATASET)
        X_train_HF, X_val_HF, y_train_HF, y_val_HF = kelas.prepareDf(df_poem_filtered)
        X_train_HF_keras, X_val_HF_keras, vocab_size, embedding_matrix = kelas.gloveTokenizer(X_train_HF, X_val_HF)
        train_dataset_HF = kelas.createDataset(X_train_HF_keras, y_train_HF)
        val_dataset_HF = kelas.createDataset(X_val_HF_keras, y_val_HF)
        model = kelas.buildModel(MODE, vocab_size, embedding_matrix)
        model.fit(
            train_dataset_HF,
            epochs = EPOCHS,
            batch_size = BATCH_SIZE,
            validation_data = val_dataset_HF,
            callbacks=[kelas.checkpointCallback(), kelas.earlyStopping()]
        )
        best_model = load_model(f'./model/best_model_{DATASET}_{TEST_SIZE}_{MODE}.keras')
        result = best_model.predict(X_val_HF_keras)
        predicted_labels = np.argmax(result, axis=1)
        accuracy = accuracy_score(y_val_HF, predicted_labels)
        print(f'Akurasi untuk skema {DATASET}, split test {TEST_SIZE*100}%, model {MODE} adalah {accuracy*100}%')
        with open('4.1. log_testing_result.txt', 'a') as f:
            f.write(f'\nAkurasi untuk skema {DATASET}, split test {TEST_SIZE*100}%, model {MODE} adalah {accuracy*100}% \n')

I0000 00:00:1744427791.818802   35401 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Epoch 1/1000


I0000 00:00:1744427796.229099   35828 cuda_dnn.cc:529] Loaded cuDNN version 90300


399/400 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.2094 - loss: 1.8586
Epoch 1: val_accuracy improved from -inf to 0.25141, saving model to ./model/best_model_deepseek_0.1_lstm.keras
400/400 ━━━━━━━━━━━━━━━━━━━━ 24s 47ms/step - accuracy: 0.2094 - loss: 1.8582 - val_accuracy: 0.2514 - val_loss: 1.5835
Epoch 2/1000
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.2532 - loss: 1.6296
Epoch 2: val_accuracy improved from 0.25141 to 0.30977, saving model to ./model/best_model_deepseek_0.1_lstm.keras
400/400 ━━━━━━━━━━━━━━━━━━━━ 18s 44ms/step - accuracy: 0.2532 - loss: 1.6296 - val_accuracy: 0.3098 - val_loss: 1.5339
Epoch 3/1000
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.2975 - loss: 1.5596
Epoch 3: val_accuracy improved from 0.30977 to 0.35056, saving model to ./model/best_model_deepseek_0.1_lstm.keras
400/400 ━━━━━━━━━━━━━━━━━━━━ 18s 46ms/step - accuracy: 0.2975 - loss: 1.5596 - val_accuracy: 0.3506 - val_loss: 1.4755
Epoch 4/1000
399/400 ━━━━━━━━━━━━━━━━━━━━ 0s 41